In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,20)


In [2]:
df1 = pd.read_csv("banglore.csv")
df1.head()


FileNotFoundError: [Errno 2] No such file or directory: 'banglore.csv'

In [ ]:
df1.groupby('area_type')['area_type'].agg('count')

In [ ]:
df2 = df1.drop(['area_type','society','balcony','availability'], axis='columns')
df2.isnull().sum()



In [ ]:
df3 = df2.dropna()
df3.isnull().sum()

In [ ]:
df3['size'].unique()

In [ ]:
df3['bhk'] = df3['size'].apply(lambda x : int(x.split(' ')[0]))
df3.head()

df3 = df3.drop(['size'],axis='columns')
df3.head()

In [ ]:
df3.head()

In [ ]:
df3['bhk'].unique()

In [ ]:
df3.total_sqft.unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

df3[~df3['total_sqft'].apply(is_float)].head(10)

In [ ]:
def convert_to_num(x):
    token = x.split('-')
    if len(token) == 2:
        return((float(token[0])+ float(token[1]))/2)
    try:
        return float(x)
    except:
        return None
    
df4 = df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_to_num)



In [ ]:
df4.loc[30]

In [ ]:
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000/df5['total_sqft']
df5.head()

In [ ]:
len(df5.location.unique())

In [ ]:
df5.location = df5.location.apply(lambda x:x.strip())
location_stats = df5.groupby('location')['location'].agg('count')
location_stats

In [ ]:
location_stats = location_stats.sort_values(ascending=False)
location_stats.head(50)

In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
df5.location = df5.location.apply(lambda x : "other" if x in location_stats_less_than_10 else x)
df5.head(10)


In [ ]:
df6 = df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape

In [ ]:
df6.price_per_sqft.describe()

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key , subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft <= (m+st))]
        df_out = pd.concat([df_out , reduced_df], ignore_index = True)
    return df_out
df7 =  remove_pps_outliers(df6)
df7.head(50)

In [ ]:
def plot_scatter_chart(df,location):
    bhk2 = df[(df.location == location) & (df.bhk==2)]
    bhk3 = df[(df.location == location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize']=(15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker= '+',color="green",label="3 BHK",s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price per square feet")
    plt.title("location")
    plt.legend()
    
plot_scatter_chart(df7 ,"Hebbal" )

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location , location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk , bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean' : np.mean(bhk_df.price_per_sqft),
                'std' : np.std(bhk_df.price_per_sqft),
                'count' : bhk_df.shape[0]
            }
            for bhk , bhk_df in location_df.groupby('bhk'):
                stats = bhk_stats.get(bhk-1)
                if stats and stats['count'] >5:
                    exclude_indices = np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

df8 = remove_bhk_outliers(df7)



In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize']=(20,10)
plt.hist(df8.price_per_sqft,rwidth = 0.8)
plt.xlabel("Price per Square Feet")
plt.ylabel("count")


In [ ]:
df8.bath.unique()

In [ ]:
df8[df8.bath>10]

In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel('Number Of Bathrooms')
plt.ylabel('Count')

In [ ]:
df8[df8.bath>df8.bhk+2]

In [ ]:
df9 = df8[df8.bath<df8.bhk+2]
df9.shape

In [ ]:
df10 = df9.drop(['size','price_per_sqft'],axis = 'columns')
df10.head(50)

In [ ]:
dummies = pd.get_dummies(df10.location)
dummies.head()

In [ ]:
df11 = pd.concat([df10,dummies.drop("other",axis='columns')],axis='columns')

In [ ]:
df11.head()

In [ ]:
df12 = df11.drop('location',axis='columns')
df12.head()

In [ ]:
X = df12.drop('price',axis='columns')
X.head()

In [ ]:
y = df12.price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits = 5 , test_size = 0.2, random_state = 0)
cross_val_score(LinearRegression(),X,y,cv=cv)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
def find_best_model(X,y):
    algos = {
      'linear_regression':{
       
        'model' : LinearRegression(),
           'Params': {
                'normalize': [True,False]
           }
        },
        'Lasso':{
                'model': Lasso(),
                'Params': {
                    'alpha': [1,2],
                    'selection': ['random','cyclic']
                }
            },
        'decision tree':{
            'model': DecisionTreeRegressor(),
            'Params':{
                'criterion': ['mse','friedman_mse'],
                'splitter' : ['best' , 'random']
            }
          }
        }
    score = []
    cv = ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    
    for algo_names, config in algos.items():
        gs = GridSearchCV(config['model'],config['Params'],cv=cv,return_train_score=False)
        gs.fit(X,y)
        score.append({
            'model': algo_names,
            'best_score' : gs.best_score_,
            'best_params' : gs.best_params_
        })
    return pd.DataFrame(score,columns=['model','best_score','best_params'])
    
find_best_model(X,y)
    

In [ ]:
X.columns

In [ ]:
def predict_price(location,sqft,bath,bhk):
    
    loc_index = np.where(X.columns==location)[0][0]
    x = np.zeros(len(X.columns))
   
    x[0]= sqft
    x[1]= bath
    x[2]= bhk
    if loc_index >= 0:
        x[loc_index]=1
        
    return lr_clf.predict([x])[0]


    

In [ ]:
predict_price('Whitefield',1000,2,2)

In [ ]:
import pickle
with open('bang.pickle','wb')as f:
    pickle.dump(lr_clf,f)

In [ ]:
import json
columns = {
    'data_columns': [col.lower() for col in X.columns]
}
with open('col.json','w')as f:
    f.write(json.dumps(columns))